### Load environment variables

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
WATSONX_APIKEY = os.environ["WATSONX_APIKEY"] = os.environ["WATSONX_API_KEY"] = os.getenv("WATSONX_APIKEY") 
WATSONX_PROJECT_ID = os.environ["WATSONX_PROJECT_ID"] = os.getenv("WATSONX_PROJECT_ID")
WATSONX_URL = os.environ["WATSONX_URL"] = os.getenv("WATSONX_URL")  

os.environ["OTEL_SDK_DISABLED"] = "true"

### Initialize our LLM 

In [ ]:
from crewai import LLM

WATSONX_MODEL_ID = "watsonx/ibm/granite-3-8b-instruct"   

llm = LLM(
    model = WATSONX_MODEL_ID,
    base_url = WATSONX_URL,
    project_id = WATSONX_PROJECT_ID,
	api_key = WATSONX_APIKEY,
    max_tokens = 10000,
    temperature = 0.1
)

### Define the Tools

- Integration with LangChain Tools

In [4]:
from crewai.tools import tool, BaseTool
from pydantic import Field
from langchain_community.tools import DuckDuckGoSearchRun

In [5]:
class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: DuckDuckGoSearchRun = Field(default_factory=DuckDuckGoSearchRun)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            return self.search.invoke(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"


### Create Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [6]:
from crewai import Agent

researcher = Agent(
    role = "Market Research Analyst",
    goal = "Provide up-to-date market analysis of the {industry} industry",
    backstory = "An expert analyst with a keen eye for industry trends and market data.",
    verbose = True,
    allow_delegation = False,
    llm = llm,
    tools = [SearchTool()]
)

writer = Agent(
    role = "Content Writer",
    goal = "Craft engaging blog posts about the {industry} sector based on provided research.",
    backstory = "A creative professional skilled in delivering impactful content.",
    verbose = True,
    allow_delegation = False,
    llm = llm,
)

### Define Tasks

In [7]:
from crewai import Task

research = Task(
    description = 'Research the latest trends in the {industry} industry and provide a summary.',
    expected_output = 'A summary of the top 2 trending developments in the {industry} industry with a unique perspective on their significance.',
    agent = researcher
)

write = Task(
    description = "Write an engaging blog post about the AI industry, based on the research analyst's summary.",
    expected_output = "A well-written blog post in markdown format, ready for publication, each section should have 1 paragraph.",
    agent = writer,
    output_file = 'new_post.md'  
)

### Create a crew of agents and execute 

- Pass the tasks to be performed by those agents.
- Note: For this example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.

In [8]:
from crewai import Crew, Process 

crew = Crew(
    agents = [researcher, writer],
    tasks = [research, write],
    process = Process.sequential,
    verbose = True
)

In [9]:
result = crew.kickoff(inputs = {"industry": "Artificial Intelligence"})

 
[2025-03-12 00:13:42][🚀 CREW 'CREW' STARTED, E47BB490-BB2A-43BA-B3D5-2CC396B72593]: 2025-03-12 00:13:42.422780
 
[2025-03-12 00:13:42][📋 TASK STARTED: RESEARCH THE LATEST TRENDS IN THE ARTIFICIAL INTELLIGENCE INDUSTRY AND PROVIDE A SUMMARY.]: 2025-03-12 00:13:42.426658
 
[2025-03-12 00:13:42][🤖 AGENT 'MARKET RESEARCH ANALYST' STARTED TASK]: 2025-03-12 00:13:42.427114
# Agent: Market Research Analyst
## Task: Research the latest trends in the Artificial Intelligence industry and provide a summary.
 
[2025-03-12 00:13:42][🤖 LLM CALL STARTED]: 2025-03-12 00:13:42.427180
 
[2025-03-12 00:13:44][✅ LLM CALL COMPLETED]: 2025-03-12 00:13:44.585500
 
[2025-03-12 00:13:44][🤖 TOOL USAGE STARTED: 'SEARCH']: 2025-03-12 00:13:44.586586
 
[2025-03-12 00:13:45][✅ TOOL USAGE FINISHED: 'SEARCH']: 2025-03-12 00:13:45.938540


# Agent: Market Research Analyst
## Thought: Thought: I need to find the latest trends in the Artificial Intelligence industry. I will use the Search tool to gather this informati

In [ ]:
from IPython.display import display, Markdown

display(Markdown(result.raw))

---

# The Future of AI: Multimodal Models and Evolving Regulations

The Artificial Intelligence (AI) industry is rapidly evolving, with two significant trends shaping its trajectory: the rise of multimodal models and AI agents, and the emergence of evolving regulatory landscapes and AI legislation.

## Multimodal Models and AI Agents: A New Era of AI Capabilities

The first trend revolves around the increasing prevalence of multimodal models and AI agents. These advanced AI systems can process and understand multiple types of data, including text, images, and audio. This capability enables more sophisticated and versatile AI applications, allowing AI systems to better understand and interact with the complex, multimodal world around us.

Multimodal models and AI agents have the potential to revolutionize various sectors, from healthcare to entertainment. For instance, in healthcare, these systems could analyze medical images and patient records to provide more accurate diagnoses. In entertainment, they could create immersive experiences by understanding and responding to user inputs across different modalities.

The significance of this trend lies in its potential to make AI systems more human-like, capable of understanding and responding to the world in a more nuanced and contextually appropriate manner. This development could pave the way for AI systems that are not just tools, but genuine collaborators and companions.

## Navigating the Regulatory Landscape: Ethical AI Use

The second trend is the evolving regulatory landscapes and AI legislation. As AI becomes more integrated into various industries and aspects of life, governments and regulatory bodies are increasingly focusing on creating guidelines and laws to ensure ethical and responsible AI use.

This trend is crucial for shaping the future of AI. It will impact how AI technologies are developed, deployed, and adopted. For instance, regulations could dictate the transparency of AI decision-making processes, ensuring users understand how AI systems arrive at their conclusions. They could also address issues like data privacy and security, ensuring that AI systems respect user data and protect it from misuse.

The evolving regulatory landscape also underscores the importance of ethical AI development. As AI systems become more sophisticated, it's essential to consider their potential societal impacts and ensure they align with human values and ethical principles.

## Conclusion: A Balanced Approach to AI's Future

The rise of multimodal models and AI agents, coupled with the evolving regulatory landscapes and AI legislation, presents both opportunities and challenges for the AI industry. While these trends promise to unlock new capabilities and applications for AI, they also necessitate careful consideration of ethical implications and responsible use.

As we move forward, it's crucial to strike a balance between fostering innovation and ensuring ethical AI use. This balance will be key to realizing the full potential of AI while mitigating potential risks and harms.

---

This blog post provides an engaging exploration of the two primary trends shaping the AI industry, offering insights into their significance and potential implications for the future of AI.

In [15]:
print(f"Tasks Output: {result.tasks_output}")
print(f"Token Usage: {result.token_usage}")

Tasks Output: [TaskOutput(description='Research the latest trends in the Artificial Intelligence industry and provide a summary.', name=None, expected_output='A summary of the top 2 trending developments in the Artificial Intelligence industry with a unique perspective on their significance.', summary='Research the latest trends in the Artificial Intelligence industry and...', raw='The top two trending developments in the Artificial Intelligence industry are:\n\n1. The rise of multimodal models and AI agents: These models can process and understand multiple types of data, such as text, images, and audio, enabling more sophisticated and versatile AI applications. This trend is significant as it allows AI systems to better understand and interact with the complex, multimodal world around us.\n\n2. Evolving regulatory landscapes and AI legislation: As AI becomes more integrated into various industries and aspects of life, governments and regulatory bodies are increasingly focusing on crea

In [ ]:
import pandas as pd

costs = 0.150 * (result.token_usage.prompt_tokens + result.token_usage.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

df_usage_metrics = pd.DataFrame([result.token_usage.dict()])
df_usage_metrics

Total costs: $0.0004


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,2789,1535,0,1254,3
